# T1069.001 - Local Groups
Adversaries may attempt to find local system groups and permission settings. The knowledge of local system permission groups can help adversaries determine which groups exist and which users belong to a particular group. Adversaries may use this information to determine which users have elevated permissions, such as the users found within the local administrators group.

Commands such as <code>net localgroup</code> of the [Net](https://attack.mitre.org/software/S0039) utility, <code>dscl . -list /Groups</code> on macOS, and <code>groups</code> on Linux can list local groups.

## Atomic Tests

### Atomic Test #1 - Permission Groups Discovery (Local)Permission Groups Discovery
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
if [ -x "$(command -v dscacheutil)" ]; then dscacheutil -q group; else echo "dscacheutil is missing from the machine. skipping..."; fi;
if [ -x "$(command -v dscl)" ]; then dscl . -list /Groups; else echo "dscl is missing from the machine. skipping..."; fi;
if [ -x "$(command -v groups)" ]; then groups; else echo "groups is missing from the machine. skipping..."; fi;
if [ -x "$(command -v id)" ]; then id; else echo "id is missing from the machine. skipping..."; fi;
if [ -x "$(command -v getent)" ]; then getent group; else echo "getent is missing from the machine. skipping..."; fi;
cat /etc/group
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 1

### Atomic Test #2 - Basic Permission Groups Discovery Windows (Local)Basic Permission Groups Discovery for Windows. This test will display some errors if run on a computer not connected to a domain. Upon execution, domain
information will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
net localgroup
net localgroup "Administrators"
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 2

### Atomic Test #3 - Permission Groups Discovery PowerShell (Local)Permission Groups Discovery utilizing PowerShell. This test will display some errors if run on a computer not connected to a domain. Upon execution, domain
information will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
get-localgroup
Get-LocalGroupMember -Name "Administrators"
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 3

### Atomic Test #4 - SharpHound3 - LocalAdmin
This module runs the Windows executable of SharpHound in order to remotely list members of the local Administrators group (SAMR)

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: SharpHound binary must exist on disk and at specified location (#{sharphound_path}).
And the computer must be domain joined (implicit authentication).

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\SharpHound.exe") { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/BloodHoundAD/BloodHound/blob/e062fe73d73c015dccb37fae5089342d009b84b8/Collectors/SharpHound.exe?raw=true" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\SharpHound.exe"

```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-Item -Path "$env:TEMP\SharpHound\" -ItemType Directory > $null
& "PathToAtomicsFolder\..\ExternalPayloads\SharpHound.exe" -d "$env:UserDnsDomain" --CollectionMethod LocalAdmin --NoSaveCache --OutputDirectory "$env:TEMP\SharpHound\"
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 4

#### Cleanup: 
```powershell
Remove-Item -Recurse $env:TEMP\SharpHound\ -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Wmic Group DiscoveryUtilizing wmic.exe to enumerate groups on the local system. Upon execution, information will be displayed of local groups on system.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic group get name
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 5

### Atomic Test #6 - WMIObject Group DiscoveryUtilizing PowerShell cmdlet - get-wmiobject, to enumerate local groups on the endpoint. Upon execution, Upon execution, information will be displayed of local groups on system.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Get-WMIObject Win32_Group
```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 6

### Atomic Test #7 - Permission Groups Discovery for Containers- Local Groups
Attackers may try to obtain a list of services that are operating on remote hosts and local network infrastructure devices, in order to identify potential vulnerabilities that can be exploited through remote software attacks. They typically use tools to conduct port and vulnerability scans in order to obtain this information.
**Supported Platforms:** containers
#### Dependencies:  Run with `sh`!
##### Description: Verify docker is installed.
##### Check Prereq Commands:
```sh
which docker

```
##### Get Prereq Commands:
```sh
if [ "" == "`which docker`" ]; then echo "Docker Not Found"; if [ -n "`which apt-get`" ]; then sudo apt-get -y install docker ; elif [ -n "`which yum`" ]; then sudo yum -y install docker ; fi ; else echo "Docker installed"; fi

```
##### Description: Verify docker service is running.
##### Check Prereq Commands:
```sh
sudo systemctl status docker  --no-pager

```
##### Get Prereq Commands:
```sh
sudo systemctl start docker

```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
docker build -t t1069 $PathtoAtomicsFolder/T1069.001/src/
docker run --name t1069_container  -d -t t1069
docker exec t1069_container ./test.sh```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 7

#### Cleanup: 
```sh
docker stop t1069_container
docker rmi -f t1069```

In [ ]:
Invoke-AtomicTest T1069.001 -TestNumbers 7 -Cleanup

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).